# 태블로 RD 업데이트

### 디렉토리 설정
- 아래 각 변수에 해당하는 알맞은 경로를 찾아 입력해주세요.
- 경로 주소는 '.../.../...' 형식을 따라주세요.

In [39]:
class directory:
    dropbox_dir = 'C:/Users/MADUP/Dropbox (주식회사매드업)'
    # 드롭박스 폴더 위치경로
    
    raw_dir = dropbox_dir + ''
    # raw_data가 적재되어 있는 폴더 위치경로
    
    download_dir = 'C:/Users/MADUP/Downloads'
    # 최종 결과파일을 다운받을 폴더 위치경로

dr = directory()

### 날짜 설정
- 보통 작일자 기준으로 요청주신 기간의 데이터를 출력하게 됩니다.
- 날짜 조정이 필요한 경우 아래 변수 값을 수정하여 출력해주세요.

In [40]:
import datetime

class report_date:
    today = datetime.date.today()
    # 오늘 날짜 ex> 2022-09-14
    
    #today = datetime.date(year = 2022, month = 9, day = 1)
    # 다른 일자를 기준으로 추출하고 싶은 경우 위 코드 수정 및 주석해제
    
    day_1 = today - datetime.timedelta(1)
    # 전일자 ex> 2022-09-13
    
    start_day = datetime.date(year=day_1.year, month=day_1.month, day=1)
    # 금월 첫 1일자 ex> 2022-09-01
    
    yearmonth = day_1.strftime('%Y%m')
    # 년월 ex> 202209
    
    month_name = str(day_1.month) + '월'
    # 금월 ex> 9월

rdate = report_date()

### 기타 설정
- 스프레드 시트 연동
- 태블로 설정 값 연동

In [41]:
class document():
    doc = spreadsheet.spread_document_read(
        'https://docs.google.com/spreadsheets/d/1efQrJpglhz0K9wKkl2QZ5VLrMu_hIUUnPG_uTSeI1Hk/edit#gid=2008544363')
    # 캠페인 정보 관리 시트 불러오기
    campaign_doc = spreadsheet.spread_document_read(
        'https://docs.google.com/spreadsheets/d/1w23AdEYLf4OJDlTElf2vwbVPttctSx-2nB51ZHQDEJQ/edit#gid=0')
    
class tableau_info():
    account_name = '쏘카'
    result_name = 'socar'

    raw_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/Tableau/RD'
    asset_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/Tableau/asset'
    result_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/Tableau/result'

스프레드시트 읽기 완료
스프레드시트 읽기 완료


### 코드
- 전처리 자동화를 위한 코드로 임의 수정을 하실 경우 복사본 생성하여 진행해주시길 바랍니다.
- 혹 아래 이미지와 같은 형식의 오류가 발생할 경우 `!pip install [패키지명]`형식의 코드 작성하여 실행해주시면 해당 패키지가 설치됩니다.
    - <img src = 'https://ifh.cc/g/ymACFT.png' align='left'>

- 이외의 오류가 발생하거나 문제가 해결되지 않는 경우, 데컨팀에 문의주세요 :)

In [42]:
# 패키지 설치 코드 (패키지 설치 오류 발생시 아래 코드 실행해주세요.)
# !pip install pyarrow
# !pip install datetime

In [43]:
import warnings
warnings.filterwarnings('ignore')

# 자동화 코드 복사 붙여넣기

---

In [44]:
import pandas as pd
import json
import os
import datetime

asset_dir = dr.dropbox_dir + '/광고사업부/데이터컨설팅/Tableau/asset'

def modify_check():
    try :
        # 수정 날짜 확인
        mtime = os.path.getmtime(asset_dir + f'/total_asset_data_{rdate.yearmonth}.csv')

        def unixtime(x):
            return datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d')

        mdate = unixtime(mtime)

        if rdate.today.strftime('%Y-%m-%d') == mdate:
            print('오늘은 이미 다른 분이 업데이트를 하셨네요')
            return 1
        else:
            print('코드 실행 감사드립니다 업데이트 시작합니다!')
            return 0
    except :
        print('코드 실행 감사드립니다 업데이트 시작합니다!')
        return 0

def facebook_read():
    ## 페이스북
    facebook_img = pd.read_csv(asset_dir + f'/facebook_ad_image_asset_daily_report_{rdate.yearmonth}.csv',
                               encoding='utf-8-sig')
    facebook_img = facebook_img.loc[pd.notnull(facebook_img['image_asset'])]
    facebook_img['소재 URL'] = facebook_img['image_asset'].apply(lambda x: json.loads(x)['url'] if x != '{}' else '')
    facebook_img['소재 유형'] = 'image'
    facebook_img = facebook_img[['owner_id', 'campaign_name', 'adset_name', 'ad_name', '소재 유형', '소재 URL', 'collected_at']]

    facebook_vid = pd.read_csv(asset_dir + f'/facebook_ad_video_asset_daily_report_{rdate.yearmonth}.csv',
                               encoding='utf-8-sig')
    facebook_vid = facebook_vid.loc[pd.notnull(facebook_vid['video_asset'])]
    facebook_vid['소재 URL'] = facebook_vid['video_asset'].apply(
        lambda x: 'https://facebook.com' + json.loads(x)['url'] if x != '{}' else '')
    facebook_vid['소재 유형'] = 'video'
    facebook_vid = facebook_vid[['owner_id', 'campaign_name', 'adset_name', 'ad_name', '소재 유형', '소재 URL', 'collected_at']]

    facebook = pd.concat([facebook_img, facebook_vid], axis=0, ignore_index=True)
    facebook = facebook.rename(columns=
                               {'campaign_name': '캠페인',
                                'adset_name': '광고그룹',
                                'ad_name': '소재'})

    facebook = facebook.sort_values(['owner_id', '캠페인', '광고그룹', '소재', '소재 유형', 'collected_at'],
                                    ascending=[True, True, True, True, False, True])
    facebook = facebook.drop_duplicates(['owner_id', '캠페인', '광고그룹', '소재'], keep='last')
    facebook = facebook[['owner_id', '캠페인', '광고그룹', '소재', '소재 유형', '소재 URL']]
    facebook['매체'] = '페이스북'
    return facebook

def kakao_read():
    ## 카카오
    kakao = pd.read_csv(asset_dir + f'/kakao_all_creative_list_{rdate.yearmonth}.csv', encoding= 'utf-8-sig')
    kakao = kakao.loc[pd.notnull(kakao['image'])]
    kakao['소재 URL'] = kakao['image'].apply(lambda x : 'https:' + json.loads(x)['url'] if x != '{}' else '')
    kakao.columns

    kakao = kakao.rename(columns = {'campaign_name' : '캠페인', 'adgroup_name' : '광고그룹','creative_name' : '소재', 'format' : '소재 유형'})
    kakao = kakao[['owner_id', '캠페인', '광고그룹', '소재', '소재 유형','소재 URL']]
    kakao = kakao.drop_duplicates(['owner_id', '캠페인', '광고그룹', '소재'])
    kakao['매체'] = '카카오'
    return kakao

def google_read():
    ## 구글
    google = pd.read_csv(asset_dir + f'/google_ads_asset_report_{rdate.yearmonth}.csv', encoding= 'utf-8-sig')
    asset_type_dict = {
    0 : 'BOOK_ON_GOOGLE',
    1 : 'CALL',
    2 : 'CALLOUT',
    3 : 'CALL_TO_ACTION',
    4 : 'DISCOVERY_CAROUSEL_CARD',
    5 : 'DYNAMIC_CUSTOM',
    6 : 'DYNAMIC_EDUCATION',
    7 : 'DYNAMIC_FLIGHTS',
    8 : 'DYNAMIC_HOTELS_AND_RENTALS',
    9 : 'DYNAMIC_JOBS',
    10 : 'DYNAMIC_LOCAL',
    11 : 'DYNAMIC_REAL_ESTATE',
    12 : 'DYNAMIC_TRAVEL',
    13 : 'HOTEL_CALLOUT',
    14 : 'IMAGE',
    15 : 'LEAD_FORM',
    16 : 'MEDIA_BUNDLE',
    17 : 'MOBILE_APP',
    18 : 'PAGE_FEED',
    19 : 'PRICE',
    20 : 'PROMOTION',
    21 : 'SITELINK',
    22 : 'STRUCTURED_SNIPPET',
    23 : 'TEXT',
    24 : 'UNKNOWN',
    25 : 'UNSPECIFIED',
    26 : 'YOUTUBE_VIDEO'
    }
    google = google.loc[pd.notnull(google['asset_image_asset_full_size_url'])]
    google['asset_type'] = google['asset_type'].apply(lambda x : asset_type_dict.get(x))
    google = google.rename(columns = {
        'campaign_name' : '캠페인',
        'ad_group_name' : '광고그룹',
        'asset_image_asset_full_size_url' : '소재 URL',
        'asset_type' : '소재 유형'})

    google = google.drop_duplicates(['캠페인', '광고그룹'])
    google['소재'] = google['광고그룹']
    google = google[['owner_id', '캠페인', '광고그룹', '소재', '소재 유형','소재 URL']]
    google['매체'] = '구글'
    return google

In [45]:
if modify_check() == 1:
    pass
else :
    facebook = facebook_read()
    kakao = kakao_read()
    google = google_read()

    asset_data = pd.concat([facebook, kakao, google], sort=False, ignore_index=True)
    asset_data.to_csv(asset_dir + f'/total_asset_data_{rdate.yearmonth}.csv', index=False, encoding='utf-8-sig')
    print('업데이트가 완료되었습니다.')

오늘은 이미 다른 분이 업데이트를 하셨네요


In [46]:
facebook = facebook_read()
kakao = kakao_read()
google = google_read()

asset_data = pd.concat([facebook, kakao, google], sort=False, ignore_index=True)
asset_data.to_csv(asset_dir + f'/total_asset_data_{rdate.yearmonth}.csv', index=False, encoding='utf-8-sig')
print('업데이트가 완료되었습니다.')

업데이트가 완료되었습니다.
